<a href="https://colab.research.google.com/github/jdu8/sentiment-analysis-comparison/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Thu Nov 17 17:10:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d kazanova/sentiment140

 84% 68.0M/80.9M [00:00<00:00, 258MB/s]
100% 80.9M/80.9M [00:00<00:00, 244MB/s]


In [3]:
! unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Dropout,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,Bidirectional,LSTM
import keras

In [5]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
columns=df.columns
columns

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [7]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.columns=['sentiment','data']
df.head()

,sentiment,data
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
y=df['sentiment']

In [10]:
df_train,df_test,y_train,y_test=train_test_split(df['data'],y,test_size=0.3)
print('DF Train Shape: ',df_train.shape)
print('DF Test Shape: ',df_test.shape)
print('Y Train Shape: ',y_train.shape)
print('Y Test Shape: ',y_test.shape)

DF Train Shape:  (1120000,)
DF Test Shape:  (480000,)
Y Train Shape:  (1120000,)
Y Test Shape:  (480000,)


In [11]:
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(df_train)
sequence_train=tokenizer.texts_to_sequences(df_train)
sequence_test=tokenizer.texts_to_sequences(df_test)

In [12]:
word2vec=tokenizer.word_index
V=len(word2vec)
print(f'Number of independent tokens in dataset: {V}')

Number of independent tokens in dataset: 543668


In [13]:
data_train=pad_sequences(sequence_train)
data_train.shape

(1120000, 50)

In [14]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape

(480000, 50)

In [23]:
D=100
EPOCHS=10
BATCH_SIZE=128

In [16]:
cnn_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Conv1D(32,3,activation='relu'),
    MaxPooling1D(3),
    Conv1D(64,3,activation='relu'),
    MaxPooling1D(3),
    Conv1D(128,3,activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
cnn_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           54366900  
                                                                 
 conv1d (Conv1D)             (None, 48, 32)            9632      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 2, 128)            2

In [17]:
lstm_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
lstm_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           54366900  
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                      

In [19]:
multi_layer_lstm_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Bidirectional(LSTM(64,return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
multi_layer_lstm_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
multi_layer_lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 100)           54366900  
                                                                 
 bidirectional_3 (Bidirectio  (None, 50, 128)          84480     
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_6 (Dropout)         (None, 64)               

In [20]:
cnn_lstm_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Conv1D(32,3,activation='relu'),
    MaxPooling1D(3),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
cnn_lstm_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_lstm_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 100)           54366900  
                                                                 
 conv1d_3 (Conv1D)           (None, 48, 32)            9632      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 16, 32)           0         
 1D)                                                             
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              49664     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)               

In [21]:
cnn_multi_layer_lstm_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Conv1D(32,3,activation='relu'),
    MaxPooling1D(3),
    Bidirectional(LSTM(64,return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
cnn_multi_layer_lstm_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_multi_layer_lstm_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 100)           54366900  
                                                                 
 conv1d_4 (Conv1D)           (None, 48, 32)            9632      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 16, 32)           0         
 1D)                                                             
                                                                 
 bidirectional_6 (Bidirectio  (None, 16, 128)          49664     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                      

In [24]:
class CNN_Saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
          self.model.save(f"model/cnn/cnn_model_{epoch}.h5")
cnn_hist=cnn_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[CNN_Saver()])

Epoch 1/10
   5/8750 [..............................] - ETA: 4:49 - loss: 1.5779 - accuracy: 0.4375

1013/8750 [==>...........................] - ETA: 4:22 - loss: 0.6513 - accuracy: 0.6198

KeyboardInterrupt: ignored

In [ ]:
class LSTM_Saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
          self.model.save(f"model/lstm/lstm_model_{epoch}.h5")
lstm_hist=lstm_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=5,batch_size=100,callbacks=[LSTM_Saver()])

In [ ]:
class Multi_Layer_LSTM_Saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
          self.model.save(f"model/multi_layer_lstm/multi_layer_lstm_model_{epoch}.h5")
multi_layer_lstm_hist=multi_layer_lstm_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[Multi_Layer_LSTM_Saver()])

In [ ]:
class CNN_LSTM_Saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
          self.model.save(f"model/cnn_lstm/cnn_lstm_model_{epoch}.h5")
cnn_lstm_hist=cnn_lstm_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[CNN_LSTM_Saver()])

In [ ]:
class CNN_Multi_Layer_LSTM_Saver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
          self.model.save(f"model/cnn_multi_layer_lstm/cnn_multi_layer_lstm_model_{epoch}.h5")
cnn_multi_layer_lstm_hist=cnn_multi_layer_lstm_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[CNN_Multi_Layer_LSTM_Saver])

In [ ]:
print(cnn_hist)

In [ ]:
print(lstm_hist)

In [ ]:
print(multi_layer_lstm_hist)

In [ ]:
print(cnn_lstm_hist)

In [ ]:
print(cnn_multi_layer_lstm_hist)